# Wykrywanie encji nazwanych z Flair

To już ostatnie laboratoria zadaniowe, w związku z tym, jeśli znajdziecie chwilę wolnego czasu, wypełnijcie proszę ankietę: https://docs.google.com/forms/d/1rHPjpL70XdXRD-ILl3AHophPNUk0AhsFus1-mtkUPsI

Pozwoli to mi poprawić laboratoria w przyszłości, z góry dziękuję :)

# Flair

Biblioteka Flair to bardzo popularne narzędzie do tagowania sekwencji. Zaintstalujmy ją

In [2]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 401 kB 4.1 MB/s 
     |████████████████████████████████| 1.6 MB 64.8 MB/s 
     |████████████████████████████████| 19.7 MB 1.2 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 788 kB 57.5 MB/s 
     |████████████████████████████████| 981 kB 59.4 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 46 kB 4.6 MB/s 
     |████████████████████████████████| 4.2 MB 44.7 MB/s 
     |████████████████████████████████| 1.2 MB 61.5 MB/s 
     |████████████████████████████████| 199 kB 69.6 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 596 kB 55.5 MB/s 
     |████████████████████████████████| 6.6 MB 20.7 MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116702 sha256=309b6625b7a852c5

In [3]:
# OPCJONALNE WSPARCIE DLA KART GRAFICZNYCH
# W colabie możemy trenować z wykorzystaniem karty graficznej, dzięki temu trening działa dużo szybciej
# Aby włączyć wsparcie dla karty graficznej musimy:
# 1. w menu 'srodowisko wykonawcze' wybrać `zmien typ srodowiska wykonawczego` i tam `akcelerator sprzętowy` = GPU
# 2. odkomentować linijki poniżej
import flair, torch
flair.device = torch.device('cuda:0') 

# Ładowanie zbioru danych i słownika z etykietami.

**Zadanie1: (1 punkt):** Stwórz słownik etykiet z wczytanego korpusu korzystając z funkcji `make_label_dictionary()`. W naszym zbiorze, etykiety do wykrycia występują w kolumnie `ner`, której identyfikator został zapisany w linijce 6. Tutorial: https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_6_CORPUS.md może okazać się pomocny.

Efektem działania powinna być lista etykiet np: 
`Dictionary with 20 tags: <unk>, Variable, Class, Application, User_Interface_Element, Code_Block, Language, Function, Data_Structure, Library, Data_Type, File_Type, File_Name, Version, HTML_XML_Tag, Device, Operating_System, Website, User_Name, Algorithm`

In [10]:
from flair.datasets import NER_ENGLISH_STACKOVERFLOW    # zbiór otagowanych postów na Stacku

corpus = NER_ENGLISH_STACKOVERFLOW().downsample(0.8)   # pobieramy korpus i zmniejszamy jego wielkość
corpus.filter_empty_sentences()                         # usuwamy puste zdania

label_type = 'ner'   # identyfikator pod którym możemy dostać typy etykiet
label_dict = corpus.make_label_dictionary(label_type)    # TODO
print('\n\nEtykiety do wykrycia')
print(label_dict)

2022-06-09 21:00:44,813 File train has 741 questions and 897 answers.
2022-06-09 21:00:44,834 File test has 249 questions and 315 answers.
2022-06-09 21:00:44,857 File dev has 247 questions and 289 answers.
2022-06-09 21:00:44,858 Reading data from /root/.flair/datasets/ner_english_stackoverflow
2022-06-09 21:00:44,862 Train: /root/.flair/datasets/ner_english_stackoverflow/train.txt
2022-06-09 21:00:44,864 Dev: /root/.flair/datasets/ner_english_stackoverflow/dev.txt
2022-06-09 21:00:44,867 Test: /root/.flair/datasets/ner_english_stackoverflow/test.txt
2022-06-09 21:00:51,763 Filtering empty sentences
2022-06-09 21:00:51,916 Corpus: 7410 train + 2349 dev + 2486 test sentences
2022-06-09 21:00:51,918 Computing label dictionary. Progress:


7410it [00:00, 49250.14it/s]

2022-06-09 21:00:52,079 Dictionary created for label 'ner' with 21 values: Application (seen 1058 times), Class (seen 1047 times), Variable (seen 903 times), User_Interface_Element (seen 772 times), Code_Block (seen 675 times), Function (seen 605 times), Library (seen 593 times), Language (seen 575 times), Data_Structure (seen 507 times), Data_Type (seen 342 times), File_Type (seen 301 times), File_Name (seen 239 times), Version (seen 223 times), HTML_XML_Tag (seen 165 times), Device (seen 163 times), Operating_System (seen 148 times), Website (seen 86 times), User_Name (seen 71 times), Algorithm (seen 31 times), Licence (seen 1 times)


Etykiety do wykrycia
Dictionary with 21 tags: <unk>, Application, Class, Variable, User_Interface_Element, Code_Block, Function, Library, Language, Data_Structure, Data_Type, File_Type, File_Name, Version, HTML_XML_Tag, Device, Operating_System, Website, User_Name, Algorithm, Licence


# Embeddingi

W narzędziu Flair możemy bardzo prosto składać ze sobą różne embeddingi. 

**Zad2 (2 punkty):** Zapoznaj się z działaniem `StackedEmbeddings` opisanego w https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_3_WORD_EMBEDDING.md i zbuduj embeddingi zawierające reprezentacje pochodzące zarówno z Glove jak i Flairowe, oparte na `news-forward`. 

In [11]:
from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings
from flair.data import Sentence

sentence = Sentence('The grass is green .')

glove_embedding = WordEmbeddings('glove')
flair_embedding_forward = FlairEmbeddings('news-forward')

embeddings = StackedEmbeddings([glove_embedding,
                               flair_embedding_forward])


embeddings.embed(sentence)

for token in sentence:
    print(token)
    print(token.embedding)

Token[0]: "The"
tensor([-3.8194e-02, -2.4487e-01,  7.2812e-01,  ..., -4.4014e-04,
        -3.9301e-02,  1.0601e-02], device='cuda:0')
Token[1]: "grass"
tensor([-8.1353e-01,  9.4042e-01, -2.4048e-01,  ..., -3.7749e-04,
        -2.3563e-02,  1.1700e-02], device='cuda:0')
Token[2]: "is"
tensor([-0.5426,  0.4148,  1.0322,  ..., -0.0061,  0.0112,  0.0100],
       device='cuda:0')
Token[3]: "green"
tensor([-0.6791,  0.3491, -0.2398,  ..., -0.0026, -0.0118,  0.0455],
       device='cuda:0')
Token[4]: "."
tensor([-3.3979e-01,  2.0941e-01,  4.6348e-01,  ..., -2.3405e-04,
         3.8688e-03,  5.7725e-03], device='cuda:0')


# Tagger i trainer

**Zadanie 3 (2 punkty)** Bazując na treściach opisanych w https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md przygotuj obiekt `SequenceTagger`, którego rozmiar wartswy ukrytej wyniesie 256. Do obiektu tego przekażemy stworzone wcześniej embeddingi, słownik `label_dict` i nazwę kolumny z etykietą ze zmiennej `label_type`. Ustawmy `use_crf` na True.

Przygotuj obiekt `ModelTrainer`, który przyjmie zarówno nasz korpus jak i stworzony przed chwilą `SequenceTagger`.

In [18]:
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer


tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type = label_type,
                        use_crf = True)    
trainer = ModelTrainer(tagger, corpus) 


#stworzony trainer możemy zacząć trenować!
trainer.train('resources/taggers/example-upos',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=5)

# a kiedy wytrenujemy, wczytujemy najlepszy model.
model = SequenceTagger.load('resources/taggers/example-upos/final-model.pt')

2022-06-09 21:19:53,587 SequenceTagger predicts: Dictionary with 81 tags: O, S-Application, B-Application, E-Application, I-Application, S-Class, B-Class, E-Class, I-Class, S-Variable, B-Variable, E-Variable, I-Variable, S-User_Interface_Element, B-User_Interface_Element, E-User_Interface_Element, I-User_Interface_Element, S-Code_Block, B-Code_Block, E-Code_Block, I-Code_Block, S-Function, B-Function, E-Function, I-Function, S-Library, B-Library, E-Library, I-Library, S-Language, B-Language, E-Language, I-Language, S-Data_Structure, B-Data_Structure, E-Data_Structure, I-Data_Structure, S-Data_Type, B-Data_Type, E-Data_Type, I-Data_Type, S-File_Type, B-File_Type, E-File_Type, I-File_Type, S-File_Name, B-File_Name, E-File_Name, I-File_Name, S-Version
2022-06-09 21:19:53,695 ----------------------------------------------------------------------------------------------------
2022-06-09 21:19:53,696 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbed

/usr/local/lib/python3.7/dist-packages/flair/trainers/trainer.py:65: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  "There should be no best model saved at epoch 1 except there "


2022-06-09 21:19:55,416 epoch 1 - iter 23/232 - loss 1.16432619 - samples/sec: 441.46 - lr: 0.100000
2022-06-09 21:19:57,152 epoch 1 - iter 46/232 - loss 0.94032945 - samples/sec: 425.40 - lr: 0.100000
2022-06-09 21:19:58,814 epoch 1 - iter 69/232 - loss 0.82986923 - samples/sec: 444.80 - lr: 0.100000
2022-06-09 21:20:00,473 epoch 1 - iter 92/232 - loss 0.77080926 - samples/sec: 444.50 - lr: 0.100000
2022-06-09 21:20:02,173 epoch 1 - iter 115/232 - loss 0.72141856 - samples/sec: 434.62 - lr: 0.100000
2022-06-09 21:20:04,059 epoch 1 - iter 138/232 - loss 0.68696591 - samples/sec: 391.05 - lr: 0.100000
2022-06-09 21:20:05,908 epoch 1 - iter 161/232 - loss 0.66625654 - samples/sec: 399.01 - lr: 0.100000
2022-06-09 21:20:07,508 epoch 1 - iter 184/232 - loss 0.64227767 - samples/sec: 461.02 - lr: 0.100000
2022-06-09 21:20:09,322 epoch 1 - iter 207/232 - loss 0.61886673 - samples/sec: 406.74 - lr: 0.100000
2022-06-09 21:20:11,031 epoch 1 - iter 230/232 - loss 0.60621236 - samples/sec: 431.65

100%|██████████| 74/74 [00:07<00:00, 10.08it/s]


2022-06-09 21:20:18,516 Evaluating as a multi-label problem: False
2022-06-09 21:20:18,546 DEV : loss 0.42523738741874695 - f1-score (micro avg)  0.1507
2022-06-09 21:20:18,773 BAD EPOCHS (no improvement): 0
2022-06-09 21:20:18,777 saving best model
2022-06-09 21:20:20,448 ----------------------------------------------------------------------------------------------------
2022-06-09 21:20:22,450 epoch 2 - iter 23/232 - loss 0.42329868 - samples/sec: 368.73 - lr: 0.100000
2022-06-09 21:20:24,105 epoch 2 - iter 46/232 - loss 0.42224534 - samples/sec: 445.66 - lr: 0.100000
2022-06-09 21:20:26,116 epoch 2 - iter 69/232 - loss 0.41555298 - samples/sec: 366.92 - lr: 0.100000
2022-06-09 21:20:27,860 epoch 2 - iter 92/232 - loss 0.40708889 - samples/sec: 423.13 - lr: 0.100000
2022-06-09 21:20:29,445 epoch 2 - iter 115/232 - loss 0.40738833 - samples/sec: 465.81 - lr: 0.100000
2022-06-09 21:20:31,141 epoch 2 - iter 138/232 - loss 0.40221479 - samples/sec: 434.95 - lr: 0.100000
2022-06-09 21:20:

...

2022-06-09 21:22:03,677 Evaluating as a multi-label problem: False
2022-06-09 21:22:03,709 DEV : loss 0.25249624252319336 - f1-score (micro avg)  0.4573


2022-06-09 21:22:03,865 BAD EPOCHS (no improvement): 0
2022-06-09 21:22:03,870 saving best model
2022-06-09 21:22:06,185 ----------------------------------------------------------------------------------------------------
2022-06-09 21:22:06,190 loading file resources/taggers/example-upos/best-model.pt
2022-06-09 21:22:06,831 SequenceTagger predicts: Dictionary with 83 tags: O, S-Application, B-Application, E-Application, I-Application, S-Class, B-Class, E-Class, I-Class, S-Variable, B-Variable, E-Variable, I-Variable, S-User_Interface_Element, B-User_Interface_Element, E-User_Interface_Element, I-User_Interface_Element, S-Code_Block, B-Code_Block, E-Code_Block, I-Code_Block, S-Function, B-Function, E-Function, I-Function, S-Library, B-Library, E-Library, I-Library, S-Language, B-Language, E-Language, I-Language, S-Data_Structure, B-Data_Structure, E-Data_Structure, I-Data_Structure, S-Data_Type, B-Data_Type, E-Data_Type, I-Data_Type, S-File_Type, B-File_Type, E-File_Type, I-File_Type,

100%|██████████| 78/78 [00:11<00:00,  6.93it/s]

2022-06-09 21:22:18,268 Evaluating as a multi-label problem: False
2022-06-09 21:22:18,306 0.4987	0.4101	0.4501	0.3417
2022-06-09 21:22:18,308 
Results:
- F-score (micro) 0.4501
- F-score (macro) 0.3987
- Accuracy 0.3417

By class:
                        precision    recall  f1-score   support

                 Class     0.4469    0.4594    0.4531       394
           Application     0.3986    0.5307    0.4553       326
              Variable     0.2735    0.2075    0.2360       294
User_Interface_Element     0.5263    0.4833    0.5039       269
        Data_Structure     0.6995    0.7435    0.7208       191
            Code_Block     0.3163    0.1216    0.1756       255
               Library     0.4692    0.2976    0.3642       205
              Function     0.5176    0.2157    0.3045       204
              Language     0.7209    0.6691    0.6940       139
             File_Name     0.6250    0.3937    0.4831       127
             File_Type     0.5208    0.4902    0.5051       102

2022-06-09 21:22:18,853 SequenceTagger predicts: Dictionary with 83 tags: O, S-Application, B-Application, E-Application, I-Application, S-Class, B-Class, E-Class, I-Class, S-Variable, B-Variable, E-Variable, I-Variable, S-User_Interface_Element, B-User_Interface_Element, E-User_Interface_Element, I-User_Interface_Element, S-Code_Block, B-Code_Block, E-Code_Block, I-Code_Block, S-Function, B-Function, E-Function, I-Function, S-Library, B-Library, E-Library, I-Library, S-Language, B-Language, E-Language, I-Language, S-Data_Structure, B-Data_Structure, E-Data_Structure, I-Data_Structure, S-Data_Type, B-Data_Type, E-Data_Type, I-Data_Type, S-File_Type, B-File_Type, E-File_Type, I-File_Type, S-File_Name, B-File_Name, E-File_Name, I-File_Name, S-Version


# Predykcja z udziałem modelu

Jeśli model został wytrenowany, poniżej znajdziemy fragment kodu, który może wykryć encje w zdaniach.

# Downsample(0.1) wykryło "Python"/Language
#Downsample(0.8) wykryło "Python"/Language, "3"/Version

In [19]:
from flair.data import Sentence

# Jeśli nasz model się wyuczył, powinien wykryć Python jako język.
# Uwaga, ponieważ pracujemy na niewielkim podzbiorze zbioru danych (downsample(0.1) próbkuje 10%), 
# otrzymywane wyniki mogą być kiepskiej jakości, najlepiej zwiększyść ilość danych 
# jeśli pracujemy w domu.
sentence = Sentence('huge files can be opened from Python 3.')   # stwórz obiekt zdania
model.predict(sentence)                                         # wykryj encje nazwane
print(sentence.to_tagged_string())                              # wyświetl zdanie i wykryte w nim encje

Sentence: "huge files can be opened from Python 3 ." → ["Python"/Language, "3"/Version]
